## Low-level benchmarks

The "low-level" benchmarks correspond to circuits composed of only one type of gate, with various "depths" (i.e. numbers of nested gates).   
For example, ```circuit-ADD-3.sheep``` contains three nested "ADD" gates: ```output=((a+b)+c)+d```.

This notebook consists of several "scans" of parameter space.  For each scan, some functions in ```benchmark_utils``` will be run to configure and run the HE libraries (this may take some time) and then there will be some analysis of the results. 

The code in this notebook should be run from the base SHEEP directory.
Dependencies are:
* pandas
* numpy
* matplotlib
* sqlalchemy

In [22]:
import os
import sys
sys.path.append(os.environ["SHEEP_HOME"])
import re

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from pysheep.low_level_benchmarks import params_for_level, run_circuit

## Scan 1 
Look at the "levelled" HE libraries (HElib_Fp and SEAL), where we can set parameters that should guarantee correct results on a certain "depth" (i.e. number of MULTIPLY operations) without the need for bootstrapping.  Loop over all gates and all input types, setting parameters appropriate for each level.

In [2]:
CIRCUIT_FILE_DIR = os.path.join(os.environ["SHEEP_HOME"],"benchmark_inputs","low_level","circuits")
## give a unique scan_id so that we can retrieve the results later on
import uuid
scan_id = str(uuid.uuid4())
## start looping over gates, input_types, depths, and contexts
for gate in ["ADD","SUBTRACT","MULTIPLY","SELECT","NEGATE"]:
    for input_type in ["bool","uint8_t","int8_t","uint16_t","int16_t","uint32_t","int32_t"]:
        for d in [1,4,8]:
            for context in ["HElib_Fp","SEAL"]:
                params = params_for_level(context,d)
                circuit_file = CIRCUIT_FILE_DIR+"/circuit-"+gate+"-"+str(d)+".sheep"
                print("Doing benchmark for %s %s %s %i" %
                        (context,gate,input_type,d))
                uploaded_OK = run_circuit(circuit_file,
                                            input_type,
                                            context,
                                            params,
                                            "serial",
                                            scan_id)
                if not uploaded_OK:
                    print("Problem running test or uploading result")
if uploaded_OK:
    print("Finished OK")

Doing benchmark for HElib_Fp ADD bool 1
Results are {'cleartext check': {'is_correct': True}, 'timings': {'evaluation': '112.200000', 'decryption': '334.400000', 'encryption': '1835.700000'}, 'outputs': {'output_0': ['0,1,0,1']}}
Doing benchmark for SEAL ADD bool 1
Results are {'cleartext check': {'is_correct': True}, 'timings': {'evaluation': '350.400000', 'decryption': '652.800000', 'encryption': '2240.000000'}, 'outputs': {'output_0': ['0,0,0,0,1,1,1,0,0,0,1,0,1,1,1,0,0,0,0,0,1,0,1,1,1,1,0,0,1,0,1,1,0,1,0,1,0,1,1,0,1,0,0,1,1,0,1,0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,0,1,1,0,1,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,0,1,1,1,1']}}
Doing benchmark for HElib_Fp ADD bool 4
Results are {'cleartext check': {'is_correct': True}, 'timings': {'evaluation': '121.000000', 'decryption': '382.000000', 'encryption': '1827.400000'}, 'outputs': {'output_0': ['0,1,0,1']}}
Doing benchmark for SEAL ADD bool 4
Results are {'cleartext check': {'is_correct': True}, 'timings': {'evaluation': '364.1

Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '434.800000', 'decryption': '618.000000', 'encryption': '2277.600000'}, 'outputs': {'output_0': ['26049,25993,36376,16701,34304,782,1165,12291,1138,17044,38550,12698,40081,10172,14972,38379,17798,8946,14465,21125,5974,37892,14058,14587,7809,3931,19912,35321,19508,14844,22621,1963,36130,2968,6874,36880,39353,15220,19184,35110,29066,12599,13401,24022,23078,5012,24601,24639,4090,39504,34451,206,33428,32934,17797,39563,2109,21886,17989,25916,18091,9206,3240,12602,31148,6961,18650,10271,40093,10793,19536,23696,24573,38851,24893,815,33369,11744,10659,26781,24958,1920,3480,11062,20451,4437,3029,26129,8573,28120,6880,16585,31214,5819,6923,4690,23714,21166,3443,33228']}}
Doing benchmark for HElib_Fp ADD uint16_t 8
Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '131.100000', 'decryption': '484.600000', 'encryption': '2253.100000'}, 'outputs': {'output_0': ['20055,44027,12943,63853']

Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '401.300000', 'decryption': '634.400000', 'encryption': '2573.000000'}, 'outputs': {'output_0': ['-1099,5163,-18499,-15757,8643,-9007,16411,-5201,14872,15100,-20040,2307,462,-8083,-17102,10584,13779,6358,516,-18699,-18123,-8567,17713,19359,-17416,-9391,4651,-2972,-3804,7058,-20096,-6264,-3488,16351,-2814,-18897,3709,-167,-6965,-10637,-2226,-18788,-10924,-15390,-3077,18064,14157,7055,8265,-16297,-20244,18682,14676,-4229,5547,5473,-20241,-14848,-7908,17220,12123,-20005,-15459,-18015,4824,2430,-1870,19769,-745,18156,-3121,-10891,-4576,-11019,10915,-3434,-8963,-1656,17098,7241,17353,14914,7726,2149,15528,8722,-14667,-10034,17716,2914,-7998,-11316,18565,20480,-13802,-696,-11646,6980,15547,-1003']}}
Doing benchmark for HElib_Fp ADD int32_t 4
Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '154.300000', 'decryption': '492.200000', 'encryption': '2804.300000'}, 'outputs': {'output_

Results are {'cleartext check': {'is_correct': True}, 'timings': {'evaluation': '389.600000', 'decryption': '608.300000', 'encryption': '2265.600000'}, 'outputs': {'output_0': ['3,-79,74,48,118,11,40,-2,88,39,18,120,72,-118,64,1,-92,27,-69,-56,3,91,-18,-12,-55,103,-47,-85,72,-60,41,90,76,117,27,39,13,-2,-70,9,-95,-26,65,66,29,85,-62,16,85,69,-119,10,-68,-102,87,-37,-32,29,-126,-27,-88,-123,85,-92,-101,-4,68,21,-124,26,-55,-100,95,-9,-77,60,39,1,92,-81,46,89,64,56,-49,34,-99,3,-42,-53,8,125,-108,-57,-56,91,-111,106,-94,20']}}
Doing benchmark for HElib_Fp SUBTRACT int8_t 8
Results are {'cleartext check': {'is_correct': True}, 'timings': {'evaluation': '138.300000', 'decryption': '435.700000', 'encryption': '2204.000000'}, 'outputs': {'output_0': ['2,-101,84,122']}}
Doing benchmark for SEAL SUBTRACT int8_t 8
Results are {'cleartext check': {'is_correct': True}, 'timings': {'evaluation': '351.000000', 'decryption': '602.200000', 'encryption': '2247.000000'}, 'outputs': {'output_0': ['47,63

Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '383.800000', 'decryption': '609.800000', 'encryption': '2398.200000'}, 'outputs': {'output_0': ['4817,26218,91,16907,24477,13620,16022,16249,10220,34535,10096,1544,16548,30484,10696,37259,22389,10784,7758,34737,12278,17375,13457,28232,7593,27852,7875,31494,11531,11099,30674,4360,17783,35445,8812,36283,21677,8079,24421,8340,22733,16270,8633,4888,35807,125,17635,10307,15924,19689,35314,6129,19269,33669,14648,18165,16915,21992,15800,28957,19695,19421,17516,1027,7980,6,39717,326,39494,34138,35522,25721,38578,15186,27774,6198,32284,29742,37010,28503,5236,36348,39486,9596,21005,10987,14488,36372,7032,17556,560,17612,13075,24208,20978,34703,37068,21683,8379,16664']}}
Doing benchmark for HElib_Fp SUBTRACT uint32_t 8
Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '158.100000', 'decryption': '487.400000', 'encryption': '2567.500000'}, 'outputs': {'output_0': ['4294965037,4294963928

...

Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '11353.600000', 'decryption': '822.200000', 'encryption': '3222.800000'}, 'outputs': {'output_0': ['1,1,0,1,0,1,1,1,1,0,0,1,1,0,1,1,0,0,1,0,1,1,0,1,1,1,1,0,0,0,1,1,0,1,1,0,0,1,0,0,1,1,1,0,0,0,0,0,1,0,1,1,0,1,0,1,1,1,1,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,1,1,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0']}}
Doing benchmark for HElib_Fp SELECT uint8_t 1
Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '3274.600000', 'decryption': '332.800000', 'encryption': '3208.900000'}, 'outputs': {'output_0': ['83,39,64,81']}}
Doing benchmark for SEAL SELECT uint8_t 1
Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '11278.800000', 'decryption': '756.700000', 'encryption': '3211.500000'}, 'outputs': {'output_0': ['163,106,202,90,179,18,99,223,127,74,144,163,226,39,27,239,83,12,127,102,68,121,234,168,192,93,249,87,222,37,249,242,126,205,186,200,68,101,116,50,48,6

Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '11423.400000', 'decryption': '749.300000', 'encryption': '3311.200000'}, 'outputs': {'output_0': ['17569,-14266,-18554,-13384,12950,2290,-16112,15623,-10506,3447,8513,-19154,-10864,-11683,20031,-7780,467,-9446,9794,-5667,-20413,-15265,8844,3748,12817,11079,3852,-10863,6980,-19820,7847,-15066,-14519,20066,11960,6461,7681,-13012,6248,-17264,-5111,7214,-16506,-5865,-12772,-1869,6551,15142,-5550,-7839,13385,-16947,-8342,-18575,20186,4464,-14661,19537,-18496,-11264,-9112,17076,9236,19235,12256,2534,-6249,1693,19442,2958,12578,-12006,-6023,-15049,-6401,19932,12013,12139,-9004,-1645,-10881,-13015,9587,3864,-7557,7319,2072,11165,4321,7545,-8636,12567,11046,7805,-16689,7349,16345,2638,15653,-15734']}}
Doing benchmark for HElib_Fp SELECT int16_t 4
Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '3293.100000', 'decryption': '334.000000', 'encryption': '3127.300000'}, 'outputs': {'out

Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '11803.400000', 'decryption': '781.700000', 'encryption': '3253.800000'}, 'outputs': {'output_0': ['-5652,-16520,-17778,20412,15007,-4509,5878,-12077,14590,13019,-14533,-18377,-18010,-9726,-6162,-6994,13374,8735,-3873,-8424,4815,14615,-3713,-19227,-13946,5011,-14820,16078,1290,18795,60,-9606,-13661,4488,-15863,6210,-497,-10791,-10278,-1880,9553,-4625,8065,6672,-5198,-10196,14832,-13990,8010,-9773,1116,3018,10142,17945,-13464,-18860,-20110,19596,-3077,2940,18867,984,-1500,-14687,-15598,10115,-19675,6373,13944,-16227,196,-18851,-12194,-13248,16785,-6200,-11873,-13266,-4718,5940,12022,-2337,-7668,-6199,-8125,1238,-1277,-1450,5401,-16035,12416,-3739,-15277,15453,-5230,-18549,9662,4854,-4832,5401']}}
Doing benchmark for HElib_Fp NEGATE bool 1
Results are {'cleartext check': {'is_correct': True}, 'timings': {'evaluation': '269.200000', 'decryption': '397.700000', 'encryption': '1066.600000'}, 'outputs': {'output

Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '193.100000', 'decryption': '622.900000', 'encryption': '1358.200000'}, 'outputs': {'output_0': ['4698,39489,10551,4923,5081,35793,14783,16288,24247,19472,16910,2799,31217,19575,19469,22721,38844,26698,27935,34949,30510,25089,30095,19354,24775,34610,28681,21994,22080,35401,28520,11320,20125,24178,24406,308,16536,30794,32638,29419,28394,17623,36752,23054,30830,32201,26003,755,5855,9019,22203,34926,32916,39001,25176,21346,18071,31881,16875,39087,38621,38874,28782,32371,14835,28892,34455,22341,15019,7128,19330,8043,37351,22923,16786,40094,13638,38777,24474,18013,27547,4610,17927,19916,318,34977,32974,13730,35701,25283,23014,29160,36654,16788,20439,3455,34998,17656,25660,4523']}}
Doing benchmark for HElib_Fp NEGATE uint16_t 8
Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '407.400000', 'decryption': '465.100000', 'encryption': '1280.300000'}, 'outputs': {'output_0': ['31678,62

Doing benchmark for SEAL NEGATE int32_t 1
Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '195.100000', 'decryption': '745.400000', 'encryption': '1312.000000'}, 'outputs': {'output_0': ['3499,3087,-7311,-4891,-7383,16903,-13223,16473,18655,-17299,9058,8890,13118,6341,2442,4218,-7070,-1153,-19446,6473,9888,286,328,-14295,18577,-10009,-2504,-15043,-18756,-3955,-19231,-1478,1202,-2609,-11543,11159,-4859,9250,11438,-19990,20370,17085,-9442,-7614,-1147,6138,5032,17112,8890,10442,-13897,7286,-14615,13277,12661,-1309,-5023,1845,-13359,702,-18767,-2108,4752,-15858,-14356,-2215,8799,-11341,-6853,-11607,-525,-3834,-1676,-5717,12515,-16980,1366,-10988,-3424,14575,15951,6813,-17052,-12242,4309,11226,-15092,10190,-20468,-14789,-5885,3496,-19884,-15534,15917,-13993,16006,10528,8248,12723']}}
Doing benchmark for HElib_Fp NEGATE int32_t 4
Results are {'cleartext check': {'is_correct': False}, 'timings': {'evaluation': '402.600000', 'decryption': '499.300000', 'encrypt

### Tabulating and plotting the results

Import some SQLAlchemy utilities from the SHEEP frontend code.

In [3]:
from pysheep.database import BenchmarkMeasurement, session
from pysheep.benchmark_utils import levels_for_params
from pysheep.low_level_benchmarks import get_gate_name_from_circuit_name

In [23]:
rows = session.query(BenchmarkMeasurement).filter_by(scan_id=scan_id).all()

The parameters and timings should now be stored in an sqlite file.
We will query this using SQLAlchemy, put the results into a Pandas DataFrame, and then make some plots.
First, define the columns we want:

In [28]:
cols = {"context":[],"gate":[],"input_bitwidth":[],"input_signed":[],"is_correct":[],"depth":[], "execution_time":[]}

Now define a couple of functions to query the database and fill the table

In [29]:
def get_levels_for_row(benchmark_measurement):
    context = benchmark_measurement.context
    param_dict = {}
    for param in benchmark_measurement.parameters:
        param_dict[param.param_name] = param.param_value
    return levels_for_param(context,param_dict)

def get_execution_time_for_row(benchmark_measurement):
    for timing in benchmark_measurement.timings:
        if timing.timing_name == "evaluation":
            return timing.timing_value

def get_depth_for_row(benchmark_measurement):
    return re.search("-([\d]+).sheep",benchmark_measurement.circuit_name).groups()[0]
        
def fill_row(benchmark_measurement, columns):
    columns["context"].append(benchmark_measurement.context)
    columns["gate"].append(get_gate_name_from_circuit_name(benchmark_measurement.circuit_name))
    columns["input_bitwidth"].append(benchmark_measurement.input_bitwidth)
    columns["input_signed"].append(benchmark_measurement.input_signed)
    columns["is_correct"].append(benchmark_measurement.is_correct)
    columns["depth"].append(get_depth_for_row(benchmark_measurement))
    columns["execution_time"].append(get_execution_time_for_row(benchmark_measurement))
    return columns

In [32]:
for row in rows:
    cols = fill_row(row,cols)
    
df = pd.DataFrame(cols)
df.head()

,context,depth,execution_time,gate,input_bitwidth,input_signed,is_correct
0,HElib_Fp,1,112.2,ADD,1,False,True
1,SEAL,1,350.4,ADD,1,False,True
2,HElib_Fp,1,121.0,ADD,1,False,True
3,SEAL,1,364.1,ADD,1,False,True
4,HElib_Fp,1,111.5,ADD,1,False,True


Now lets plot the execution time vs depth for each gate type for 8-bit integers


In [65]:
def execution_time_vs_op_depth(input_dataframe, gate_name, bitwidth, is_signed, libs_list):
    print("Length of original df {}".format(df.shape))
    rows = input_dataframe[(input_dataframe.input_bitwidth==bitwidth) & 
                     #      ((input_dataframe.depth == 1)| 
                      #      (input_dataframe.depth == 4)|
                       #     (input_dataframe.depth == 8)) #& 
                           (input_dataframe.gate==gate_name) & 
                           input_dataframe.context.isin(libs_list) & 
                           (input_dataframe.input_signed==is_signed)
                          ]
    print(" new length {}".format(rows.shape))
 #   return rows
    return rows.drop_duplicates(["context", "depth"]).pivot(
        index='context', columns='depth', values='execution_time'
    )

In [69]:
new_rows=execution_time_vs_op_depth(df,"ADD",8,False,["HElib_Fp","SEAL"])
new_rows


Length of original df (420, 7)
 new length (12, 7)


depth,1
context,
HElib_Fp,136.4
SEAL,348.2


In [71]:
for row in rows:
    print(row.circuit_name)

circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-ADD-1.sheep
circuit-SUBTRACT-1.sheep
circuit-SUBTRACT-1.sheep
circuit-SUBTRACT-1.sheep
circuit-SUBTRACT-1.sheep
circuit-SUBTRACT-1.sheep
circuit-SUBTRACT-1.sheep
circuit-SU

In [ ]:
all_ops = set(all_rows.gate_name)
bitwidth = 8
is_signed = True
for i, op in enumerate(all_ops):
    if op == "COMPARE": continue 
    libs = ["HElib_Fp", "SEAL"]
    rows = libs_op_levels(op, bitwidth, is_signed, libs)
    axis = plt.subplot((len(all_ops) + 2 // 2) // 2, 2, i + 1)
#     axis.set_yscale("log")
    rows.plot(ax=axis, title=op + ' with bitwidth = ' + str(bitwidth), kind='bar', figsize=(15, 10), legend=False if i!=0 else True)
plt.tight_layout()
plt.show()

### Scan 2: 
Set parameters for the "leveled" libraries (HElib_Fp and SEAL) to be able to support multiplicative depth of 4, then see how many of each operation we can do and still get the correct results.

In [ ]:
def libs_op_depth_level4(op_str, bitwidth, is_signed, libs_list):
    rows = all_rows[(all_rows.is_correct==True) & (all_rows.Levels==4) & (all_rows.input_bitwidth==bitwidth) & (all_rows.gate_name==op_str) & all_rows.context_name.isin(libs_list) & (all_rows.input_signed==is_signed)]
    rows.context_name.name = "Library"
    rows.execution_time.name = "Time (s)" 
    return rows.drop_duplicates(["context_name", "depth"]).pivot(index='context_name', columns='depth', values='execution_time')
all_ops = set(all_rows.gate_name)

In [ ]:

all_ops = ["ADD","SUBTRACT","MULTIPLY","NEGATE"]
bitwidth = 8
is_signed = False
for i, op in enumerate(all_ops):
    if op == "COMPARE": continue 
    libs = ["HElib_Fp", "SEAL"]
    rows = libs_op_depth_level4(op, bitwidth, is_signed, libs)
    
    axis = plt.subplot((len(all_ops) + 2 // 2) // 2, 2, i + 1)
    #axis.set_yscale("log")
    rows.plot(ax=axis, title=op + ' with bitwidth = ' + str(bitwidth), kind='bar', figsize=(15, 10), legend=False if i!=0 else True)
plt.tight_layout()
plt.show()

### Scan 3:
chain $d$ gates together, with the leveled libraries set to the appropriate parameters to support that depth.


HElib_F2 and TFHE both deal with binary arrays, and by default bootstrap for every operation:

In [ ]:
def libs_op_depth(op_str, bitwidth, is_signed, libs_list):
    rows = all_rows[(all_rows.input_bitwidth==bitwidth) & (all_rows.gate_name==op_str) & all_rows.context_name.isin(libs_list) & (all_rows.input_signed==is_signed)]
    rows.context_name.name = "Library"
    rows.execution_time.name = "Time (s)" 
    return rows.drop_duplicates(["context_name", "depth"]).pivot(index='context_name', columns='depth', values='execution_time')

In [ ]:
all_ops = set(all_rows.gate_name)
bitwidth = 8
is_signed = False
for i, op in enumerate(all_ops):
    if op == "COMPARE": continue 
    libs = ["HElib_F2", "TFHE"]# all_libs
    rows = libs_op_depth(op, bitwidth, is_signed, libs)
    axis = plt.subplot((len(all_ops) + 2 // 2) // 2, 2, i + 1)
    #axis.set_yscale("log")
    rows.plot(ax=axis, title=op + ' with bitwidth = ' + str(bitwidth), kind='bar', figsize=(15, 10), legend=False if i!=0 else True)
plt.tight_layout()
plt.show()

Now we do the same for the two libraries that operate on integers - SEAL and HElib_Fp

In [ ]:
all_ops = set(all_rows.gate_name)
bitwidth = 8
is_signed = False
for i, op in enumerate(all_ops):
    if op == "COMPARE": continue 
    libs = ["HElib_Fp", "SEAL"]# all_libs
    rows = libs_op_depth(op, bitwidth, is_signed, libs)
    axis = plt.subplot((len(all_ops) + 2 // 2) // 2, 2, i + 1)
    #axis.set_yscale("log")
    rows.plot(ax=axis, title=op + ' with bitwidth = ' + str(bitwidth), kind='bar', figsize=(15, 10), legend=False if i!=0 else True)
plt.tight_layout()
plt.show()

### Scan 4: 
Look at how the number of bits in the input type affects the execution time for a single gate of each operation.

In [ ]:
def ops_lib_bitwidth(depth, is_signed, lib, ops):
    rows = all_rows[ (all_rows.context_name==lib)&(all_rows.input_signed==is_signed)&(all_rows.depth==depth)&(all_rows.gate_name.isin(ops))]
    rows.execution_time.name = "Time (s)" 
    rows.gate_name.name = "Operation"
    return rows.drop_duplicates(["gate_name", "input_bitwidth"]).pivot(index='gate_name', columns='input_bitwidth', values='execution_time')

In [ ]:
rows = ops_lib_bitwidth(1,False,"TFHE",["ADD"])
rows

In [ ]:
all_ops = set(all_rows.gate_name)
ops = all_ops#["ADD", "SUBTRACT"]
ops = ["ADD","SUBTRACT","MULTIPLY"]
all_libs = ["HElib_F2", "HElib_Fp", "TFHE", "SEAL"]
depth = 1
is_signed = False
for i, lib in enumerate(all_libs):
    rows = ops_lib_bitwidth(depth, False, lib, ops)
    axis = plt.subplot((len(all_libs) + 2 // 2) // 2, 2, i + 1)
    #axis.set_yscale("log")
    rows.plot(ax=axis, title='{} at depth {} ({}signed)'.format(lib, depth, '' if is_signed else 'un'), kind='bar', figsize=(15, 10), legend=False if i!=0 else True)
plt.tight_layout()
plt.show()